In [1]:
import pandas as pd

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers


/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [40]:
test_data = pd.read_csv('test_data.csv')
train_data = pd.read_csv('train_data.csv')
train_labels = pd.read_csv('train_labels.csv')

In [41]:
del train_data['is_duplicate']

In [42]:
train_data_labels = pd.merge(train_data,train_labels, how='left')

In [43]:
train_data_labels = train_data_labels.dropna()

In [45]:
train_data_labels['question1'] = train_data_labels['question1'].str.replace('[^\w\s]','')
train_data_labels['question2']= train_data_labels['question2'].str.replace('[^\w\s]','')

In [46]:
train_data_labels.head()

,id,question1,question2,is_duplicate
0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,What is the story of Kohinoor KohiNoor Diamond,What would happen if the Indian government sto...,0
2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,Why am I mentally very lonely How can I solve it,Find the remainder when math2324math is divide...,0
4,4,Which one dissolve in water quikly sugar salt ...,Which fish would survive in salt water,0


In [48]:
train_data_labels['question1'] = train_data_labels['question1'].apply(lambda x: re.sub(r"n't", " not ", x))
train_data_labels['question2'] = train_data_labels['question2'].apply(lambda x: re.sub(r"n't", " not ", x))

In [49]:
train_data_labels['question1'] = train_data_labels['question1'].apply(lambda x:re.sub(r"\'ve", " have ", x))
train_data_labels['question1'] = train_data_labels['question1'].apply(lambda x:re.sub(r"can't", "cannot ", x))
train_data_labels['question1'] = train_data_labels['question1'].apply(lambda x:re.sub(r"i'm", "i am ", x))
train_data_labels['question1'] = train_data_labels['question1'].apply(lambda x:re.sub(r"\'re", " are ", x))
train_data_labels['question1'] = train_data_labels['question1'].apply(lambda x:re.sub(r"\'d", " would ", x))
train_data_labels['question1'] = train_data_labels['question1'].apply(lambda x:re.sub(r"\'ll", " will ", x))

In [50]:
train_data_labels['question2'] = train_data_labels['question2'].apply(lambda x:re.sub(r"\'ve", " have ", x))
train_data_labels['question2'] = train_data_labels['question2'].apply(lambda x:re.sub(r"can't", "cannot ", x))
train_data_labels['question2'] = train_data_labels['question2'].apply(lambda x:re.sub(r"i'm", "i am ", x))
train_data_labels['question2'] = train_data_labels['question2'].apply(lambda x:re.sub(r"\'re", " are ", x))
train_data_labels['question2'] = train_data_labels['question2'].apply(lambda x:re.sub(r"\'d", " would ", x))
train_data_labels['question2'] = train_data_labels['question2'].apply(lambda x:re.sub(r"\'ll", " will ", x))

In [51]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [52]:
train_data_labels['question1'] = train_data_labels['question1'].str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in stop))
train_data_labels['question2'] = train_data_labels['question2'].str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in stop))

In [53]:
train_data_labels['question1'] = train_data_labels['question1'].apply(lambda x: " ".join(x.lower() for x in x.split()))
train_data_labels['question2'] = train_data_labels['question2'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [54]:
from nltk.tokenize import word_tokenize
import nltk


train_data_labels['question1'] = train_data_labels['question1'].apply(word_tokenize)
train_data_labels['question2'] = train_data_labels['question2'].apply(word_tokenize)

In [55]:
train_data_labels.head()

,id,question1,question2,is_duplicate
0,0,"[what, step, step, guide, invest, share, marke...","[what, step, step, guide, invest, share, market]",0
1,1,"[what, story, kohinoor, kohinoor, diamond]","[what, would, happen, indian, government, stol...",0
2,2,"[how, i, increase, speed, internet, connection...","[how, internet, speed, increased, hacking, dns]",0
3,3,"[why, i, mentally, lonely, how, i, solve]","[find, remainder, math2324math, divided, 2423]",0
4,4,"[which, one, dissolve, water, quikly, sugar, s...","[which, fish, would, survive, salt, water]",0


In [56]:
question1 = train_data_labels['question1'].tolist()

In [57]:
question2 = train_data_labels['question2'].tolist()

In [58]:
questions = question1 + question2

In [59]:
import gensim 

In [60]:
model = gensim.models.Word2Vec(questions, min_count=1,size=100,workers=4)

In [61]:
model.save('QuoraGensim_v1.bin')